In [1]:
import sys, os, subprocess
repo_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(1, os.path.join(sys.path[0], repo_dir))
from helper.HolmesReader import HolmesReader

holmesReader = HolmesReader()
collections = holmesReader.get_collections(as_dataframe=True)

In [22]:
a = [i for i in collections['text']]

/Users/arthur/.pyenv/versions/3.8.6/envs/default/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=5, stop_words='english')
dtm = cv.fit_transform(a)

/Users/arthur/.pyenv/versions/3.8.6/envs/default/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
from sklearn.decomposition import LatentDirichletAllocation
# Build LDA Model with GridSearch params
lda_model = LatentDirichletAllocation(n_components=5,
                                      learning_decay=0.5,         
                                      max_iter=50,                
                                      learning_method='online',   
                                      random_state=42,            
                                      batch_size=5000,            
                                      evaluate_every = -1,        
                                      n_jobs = -1)                
lda_output = lda_model.fit_transform(dtm)

/Users/arthur/.pyenv/versions/3.8.6/envs/default/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

/Users/arthur/.pyenv/versions/3.8.6/envs/default/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


THE TOP 15 WORDS FOR TOPIC #0
['come', 'friend', 'night', 'face', 'say', 'little', 'watson', 'did', 'room', 'time', 'know', 'man', 'mr', 'said', 'holmes']


THE TOP 15 WORDS FOR TOPIC #1
['sir', 'think', 'came', 'door', 'say', 'face', 'way', 'know', 'did', 'room', 'watson', 'mr', 'man', 'said', 'holmes']


THE TOP 15 WORDS FOR TOPIC #2
['house', 'came', 'face', 'think', 'time', 'did', 'know', 'come', 'little', 'room', 'watson', 'mr', 'man', 'said', 'holmes']


THE TOP 15 WORDS FOR TOPIC #3
['hand', 'face', 'time', 'did', 'know', 'room', 'way', 'shall', 'watson', 'little', 'come', 'mr', 'man', 'holmes', 'said']


THE TOP 15 WORDS FOR TOPIC #4
['mr', 'shall', 'watson', 'hopkins', 'face', 'room', 'case', 'night', 'little', 'inspector', 'colonel', 'horse', 'man', 'holmes', 'said']




In [26]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, dtm, cv, mds='tsne')
panel

/Users/arthur/.pyenv/versions/3.8.6/envs/default/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/arthur/.pyenv/versions/3.8.6/envs/default/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      150.222153  -72.269554       1        1  95.487534
4       51.163364 -159.968018       2        1   4.490016
0      -73.144554   21.523815       3        1   0.007483
1      -66.410133 -110.606438       4        1   0.007483
3       54.417324   12.443444       5        1   0.007483, topic_info=        Term         Freq        Total Category  logprob  loglift
1817  holmes  2435.000000  2435.000000  Default  30.0000  30.0000
3184    said  2087.000000  2087.000000  Default  29.0000  29.0000
2309     man  1473.000000  1473.000000  Default  28.0000  28.0000
2235  little   740.000000   740.000000  Default  27.0000  27.0000
3138    room   780.000000   780.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
1844   house     0.002662   601.676513   Topic5  -8.3447  -2.8280
2340  matter     0.002662   439.593728   Topic5  -8.3449  -2.5143
521     case     0.002662   526.483726   Topic5  -8.3449  -2.6947
4217     yes     0.002662   502.437364   Topic5  -8.3449  -2.6479
2949   quite     0.002662   324.499768   Topic5  -8.3449  -2.2108

[386 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
125       1  0.998233  american
181       1  0.767028    arouse
181       2  0.191757    arouse
251       1  0.766605   average
251       2  0.191651   average
...     ...       ...       ...
4210      1  0.999023      yard
4217      1  0.965294       yes
4217      2  0.035825       yes
4220      1  0.992136     young
4220      2  0.008938     young

[333 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 2, 4])

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
vis_nmf = pyLDAvis.sklearn.prepare(clf, vectors, vectorizer)